In [1]:
import os
import numpy as np
import time
import cv2
import shutil
from matplotlib import pyplot as plt
from tqdm.auto import tqdm

from IPython import display
from PIL import Image
from ultralytics import YOLO
from PIL import ImageFont, ImageDraw, Image

In [2]:
model = YOLO('/Users/juhyeon/python-workspace/ROADs/best.pt')

In [3]:
label2id = {'etc': 0,
            'PE드럼 정상': 1,
            'PE드럼 파손': 2,
            'PE방호벽 정상': 3,
            'PE방호벽 파손': 4,
            'PE안내봉 정상': 5,
            'PE안내봉 파손': 6,
            '라바콘 정상': 7,
            '라바콘 파손': 8,
            '시선유도봉 정상': 9,
            '시선유도봉 파손': 10,
            '제설함 정상': 11,
            '제설함 파손': 12,
            'PE입간판 정상': 13,
            'PE입간판 파손': 14,
            'PE휀스 정상': 15,
            'PE휀스 파손': 16}

# 반대로도 매핑할 수 있도록 딕셔너리 뒤집기!
id2label = {v: k for k, v in label2id.items()}

In [ ]:
def find_largest_bbox(track_info_dict, id2label):
    """
    각 bounding box의 넓이를 계산하고, 가장 큰 bbox를 찾는 함수
    """
    max_area = 0
    max_area_bbox = None
    max_area_track_id = None
    max_area_class_name = None

    for track_id, track_info in track_info_dict.items():
        class_id = track_info['cls_id']
        class_name = id2label[class_id] if class_id in id2label else f'Unknown_{class_id}'

        bbox = track_info['bbox'].int().tolist()
        area = (bbox[2] - bbox[0]) * (bbox[3] - bbox[1])

        if area > max_area:
            max_area = area
            max_area_bbox = bbox
            max_area_track_id = track_id
            max_area_class_name = class_name

    return max_area_bbox, max_area_track_id, max_area_class_name

def crop_and_save(frame, bbox, path):
    """
    주어진 bbox를 프레임에서 잘라내고, 잘라낸 부분을 주어진 경로에 저장하는 함수
    """
    cropped_frame = frame[bbox[1]:bbox[3], bbox[0]:bbox[2]]
    cv2.imwrite(path, cropped_frame)
    
def is_overlapping(small_img_bbox, detected_bbox):
    """
    크롭이미지와 이번에 검출된 이미지 사이에 겹치는 영역이 있는지 확인
    """
    x_overlap = max(0, min(small_img_bbox[2], detected_bbox[2]) - max(small_img_bbox[0], detected_bbox[0]))
    y_overlap = max(0, min(small_img_bbox[3], detected_bbox[3]) - max(small_img_bbox[1], detected_bbox[1]))
    overlap_area = x_overlap * y_overlap
    
    return overlap_area > 0

In [ ]:
def transfer_data(cursor, conn, s3_client, bucket_name, local_image_path, category_name):
    response = requests.get('https://ipinfo.io')
    data = response.json()
    location = data.get('loc')
    if location:
        lat, long = location.split(',')
    else:
        print("위치 정보를 가져올 수 없습니다.")

    current_datetime = datetime.now()
    s3_object_key = f'finalprojectimage/{os.path.basename(local_image_path)}'
    s3_url = f'https://{bucket_name}.s3.ap-northeast-2.amazonaws.com/{s3_object_key}'

    # 'gps_data' 테이블에 GPS 정보, 날짜, S3 경로, 카테고리 저장
    query = 'INSERT INTO gps_data (latitude, longitude, datetime, img_path, category) VALUES (%s, %s, %s, %s, %s)'
    values = (lat, long, current_datetime, s3_url, category_name)
    
    # s3에 이미지 업로드
    s3_client.upload_file(local_image_path, bucket_name, s3_object_key)
    print(f"Image uploaded to S3: {s3_object_key}")

    cursor.execute(query, values)
    conn.commit()

In [ ]:
import psycopg2
from geopy.geocoders import Nominatim
from decimal import Decimal
from datetime import datetime
import requests
import boto3

# AWS 계정 정보
aws_access_key_id = 
aws_secret_access_key = 
aws_region = 
bucket_name = 

# AWS S3 클라이언트 생성
s3_client = boto3.client('s3', 
                         aws_access_key_id=aws_access_key_id,
                         aws_secret_access_key=aws_secret_access_key, 
                         region_name=aws_region)

In [9]:
##### setting #####
image_folder_path = '/Users/juhyeon/python-workspace/ROADs/test'
output_folder_path = '/Users/juhyeon/python-workspace/ROADs/test_output_frames'

test_paths = []
for filename in sorted(os.listdir(image_folder_path), key=lambda x: int(x.split("_")[1].split(".")[0])):
    if filename.endswith(".jpg") and filename.startswith("frame_"):
        file_path = os.path.join(image_folder_path, filename)
        test_paths.append(file_path)

broken_categories = [2, 4, 6, 8, 10, 12, 14, 16]   # 파손만 검출해야함
font = ImageFont.truetype('/Users/juhyeon/python-workspace/ROADs/NanumGothic.ttf', 30)

tracks = {}
current_frame_track_info= {}

IMGSZ = 640
prev_cropped_frame = None
small_img_bbox = [50, 50, 450, 350]  # 좌측상단의 크롭된 이미지 위치와 크기를 표현하는 bbox
####################

##### PostgreSQL DB 연결 #####
db_params = {'dbname': 'test',
             'user': 'postgres',
             'password': 'postgres',
             'host': 'localhost',
             'port': '5432'
            }

conn = psycopg2.connect(**db_params)
cursor = conn.cursor()
#############################

#####   1. 이미지를 순차적으로 1~100 돌면서 검출 & 프레임당 가지고 있는 정보 가져오기(tracks에 저장됨)  #####
for i, image_path in enumerate(test_paths):
    frame = cv2.imread(image_path)
    results = model.track(frame, persist=True, imgsz=IMGSZ, verbose=False)
    # 트랙: {0: {'latest_bbox_info': {'bbox': tensor([550.9120, 516.0292, 591.8289, 719.9888]), 'cls_id': 0}}}
    print(f'트랙: {tracks}')
    
    current_frame_track_info = {int(box[4].item()): {'bbox': box[:4], 
                                                 'cls_id': int(box[6].item()) if len(box) == 7 else int(box[5].item())} 
                            for box in results[0].boxes.data if int(box[6].item() if len(box) == 7 else box[5].item()) in broken_categories}
    print(f'current_frame_track_info: {current_frame_track_info}')
    
#####   2. tracks{}에 저장된 각각의 track_id 별로 bbox 넓이 비교해서 제일 큰 것만 남도록 하는 과정   #####
    
    ######## [추가!!!] 각 프레임에서 가장 큰 bbox와 해당하는 트랙 ID, 클래스명 찾기
    largest_bbox, largest_bbox_track_id, largest_bbox_class_name = find_largest_bbox(current_frame_track_info, id2label)
    ##################################################################
    
    # 각 프레임에서 검출된 모든 객체의 bbox와 클래스명을 화면에 표시
    for track_id, track_info in current_frame_track_info.items():
        bbox = track_info['bbox'].int().tolist()
        cls_id = track_info['cls_id']
        category_name = id2label[cls_id] if cls_id in id2label else f'Unknown_{cls_id}'
        
        if track_id in tracks:
            # 기존 정보가 없다면?
            if 'latest_bbox_info' not in tracks[track_id]:
                tracks[track_id]['latest_bbox_info'] = track_info
                tracks[track_id]['frame_i'] = i
                
            # 기존 정보가 있다면 비교!
            else:
                current_bbox_size = (track_info['bbox'][2] - track_info['bbox'][0]) * (track_info['bbox'][3] - track_info['bbox'][1])
                stored_bbox_size = (tracks[track_id]['latest_bbox_info']['bbox'][2] - tracks[track_id]['latest_bbox_info']['bbox'][0]) * \
                                   (tracks[track_id]['latest_bbox_info']['bbox'][3] - tracks[track_id]['latest_bbox_info']['bbox'][1])
                
                # 현재 bbox 넓이가 더 크면 원래 정보를 업데이트
                #  track_info : {'bbox': tensor([174.2584, 524.3750, 223.3572, 684.6147]), 'cls_id': 0}
                if current_bbox_size > stored_bbox_size:
                    tracks[track_id]['latest_bbox_info'] = track_info
                    tracks[track_id]['frame_i'] = i  # 가장 큰 bbox를 가진 프레임 정보 업데이트
        else:
            # 새로운 트랙 ID일 경우, 현재 프레임 정보 저장
            tracks[track_id] = {'latest_bbox_info': track_info, 'frame_i': i}
        ######################################################################
        
    ################################ [추가!!!]검출된 bbox들이 좌측상단의 이미지와 겹치는지 확인 ##########
        if not is_overlapping(small_img_bbox, bbox):
            # 겹치지 않으면 bbox를 그림
            cv2.rectangle(frame, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 255, 0), 3)
            
            ##### 한글처리 위해서 CV => PIL => CV 변환과정 진행 #####
            frame_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            draw = ImageDraw.Draw(frame_pil)
            draw.text((bbox[0], bbox[1] - 30), category_name, font=font, fill=(0, 255, 0))
            
            frame = cv2.cvtColor(np.array(frame_pil), cv2.COLOR_RGB2BGR)
        
    # 좌측상단에 띄워주기 (선택된 bbox 잘라서!)
    if largest_bbox is not None and label2id[largest_bbox_class_name] in broken_categories:
        cropped_frame = frame[largest_bbox[1]:largest_bbox[3], largest_bbox[0]:largest_bbox[2]]
        prev_cropped_frame = cropped_frame.copy()    
        
        # 이전 프레임의 cropped 부분을 현재 프레임의 좌측 상단에 붙이기
        if prev_cropped_frame is not None:
            # (가로, 세로)
            resized_prev_cropped_frame = cv2.resize(prev_cropped_frame, (400, 300))
            # 현재 프레임의 좌측 상단 부분에 이전 프레임의 cropped 부분 붙임 (세로, 가로)
            frame[50:350, 50:450] = resized_prev_cropped_frame
    ##########################################################################################

    cv2.imshow('Frame', frame)
    cv2.waitKey(1)  # 이미지를 1ms 동안 표시. 0을 입력하면 키 입력이 있을 때까지 이미지를 계속 표시

    
#####   3. 더 이상 다음 프레임에 해당 track_id가 검출되지 않는다면, tracks{}에 남아있는 bbox, fram_id 정보로 이미지 저장 #####

    # 이전 프레임에서 트래킹했던 track_ID 중에서 현재 프레임에서는 검출되지 않은 track_id를 찾아냄
    disappeared_track_ids = set(tracks.keys()) - set(current_frame_track_info.keys())
    
    # 더 이상 현재 프레임에서 검출되지 않는 track_id 에서 작업 수행
    for disappeared_track_id in disappeared_track_ids:
        try:
            latest_bbox_info = tracks[disappeared_track_id]['latest_bbox_info']
            output_path = os.path.join(output_folder_path, f'frame_{tracks[disappeared_track_id]["frame_i"]}_track_{disappeared_track_id}.jpg')
            
            # [xmin, ymin, xmax, ymax] 정수 coordinate
            bbox = latest_bbox_info['bbox'].int().tolist()
            
            cls_id = latest_bbox_info['cls_id']
            category_name = id2label[cls_id] if cls_id in id2label else f'Unknown_{cls_id}'
            
            # 가장 큰 bbox를 가진 프레임의 이미지를 다시 불러오기
            img_path = test_paths[tracks[disappeared_track_id]["frame_i"]]
            frame = cv2.imread(img_path)
            
            # (bbox[0], bbox[1]): 왼쪽 상단 좌표 (xmin, ymin)
            # (bbox[2], bbox[3]): 오른쪽 하단 좌표 (xmax, ymax)
            cv2.rectangle(frame, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 255, 0), 3)
            
            # 한글처리 위해서 CV => PIL => CV 변환과정 진행
            frame_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            draw = ImageDraw.Draw(frame_pil)
            draw.text((bbox[0], bbox[1] - 30), category_name, font=font, fill=(0, 255, 0))
            frame = cv2.cvtColor(np.array(frame_pil), cv2.COLOR_RGB2BGR)
            cv2.imwrite(output_path, frame)

            print(f'Saved: {output_path} for Track ID: {disappeared_track_id}')
            
            # DB/S3 으로 데이터(좌표,날짜,이미지,url)전송
            transfer_data(cursor, conn, s3_client, bucket_name, output_path, category_name)

            # 해당 트랙을 더 이상 추적하지 않도록 목록에서 제거
            del tracks[disappeared_track_id]
            
        except Exception as e:
            print(f"Error occurred for Track ID {disappeared_track_id} on frame {i}: {e}")

# 프레임 보여주던 창 닫기!
cv2.destroyAllWindows()

#####    4.  프레임이 모두 재생되었는데, 마지막 프레임에 끝까지 검출되는 것이 있다면?   #####
# tracks{} 에서, 마지막까지 남은 track_id의 정보를 바탕으로 이미지 저장하고 끝냄
for track_id in tracks.keys():
    try:
        latest_bbox_info = tracks[track_id]['latest_bbox_info']
        output_path = os.path.join(output_folder_path, f'frame_{tracks[track_id]["frame_i"]}_track_{track_id}.jpg')
        
        bbox = latest_bbox_info['bbox'].int().tolist()
        
        cls_id = latest_bbox_info['cls_id']
        category_name = id2label[cls_id] if cls_id in id2label else f'Unknown_{cls_id}'
        
        img_path = test_paths[tracks[track_id]["frame_i"]]
        frame = cv2.imread(img_path)
        
        cv2.rectangle(frame, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 255, 0), 3)
        
        ##### 한글처리 위해서 CV => PIL => CV 변환과정 진행 #####
        frame_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        draw = ImageDraw.Draw(frame_pil)
        draw.text((bbox[0], bbox[1] - 30), category_name, font=font, fill=(0, 255, 0))
        frame = cv2.cvtColor(np.array(frame_pil), cv2.COLOR_RGB2BGR)
        cv2.imwrite(output_path, frame)
        print(f'Saved: {output_path} for Track ID: {track_id}')
        
        # DB/S3 으로 데이터(좌표,날짜,이미지,url)전송
        transfer_data(cursor, conn, s3_client, bucket_name, output_path, category_name)

    except Exception as e:
        print(f"Error occurred for Track ID {track_id} on the final frame : {e}")

cursor.close()
conn.close()

/Users/juhyeon/python-workspace/ROADs/test/frame_1.jpg
트랙: {}
current_frame_track_info: {1: {'bbox': tensor([1106.0850,  188.5902, 1250.5116,  264.6689]), 'cls_id': 14}}
/Users/juhyeon/python-workspace/ROADs/test/frame_2.jpg
트랙: {1: {'latest_bbox_info': {'bbox': tensor([1106.0850,  188.5902, 1250.5116,  264.6689]), 'cls_id': 14}, 'frame_i': 0}}
current_frame_track_info: {1: {'bbox': tensor([1110.3380,  190.9170, 1254.2474,  270.7471]), 'cls_id': 14}}
/Users/juhyeon/python-workspace/ROADs/test/frame_3.jpg
트랙: {1: {'latest_bbox_info': {'bbox': tensor([1110.3380,  190.9170, 1254.2474,  270.7471]), 'cls_id': 14}, 'frame_i': 1}}
current_frame_track_info: {1: {'bbox': tensor([1113.1306,  197.0028, 1258.3767,  277.9191]), 'cls_id': 14}}
/Users/juhyeon/python-workspace/ROADs/test/frame_4.jpg
트랙: {1: {'latest_bbox_info': {'bbox': tensor([1113.1306,  197.0028, 1258.3767,  277.9191]), 'cls_id': 14}, 'frame_i': 2}}
current_frame_track_info: {1: {'bbox': tensor([1117.9114,  201.6272, 1266.4486,  28